In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import time

In [12]:
def Fangraphs_teams_season(start,end,table_type):
    seasons = np.arange(start,end+1,1)
    

    driver = webdriver.Chrome()
    
    df_list=[]
    
    for season in seasons:
        url=f"https://www.fangraphs.com/leaders/major-league?pos=all&stats=bat&lg=all&qual=0&type={table_type}&month=0&ind=0&team=0%2Cts&rost=&age=&filter=&players=0&startdate=&enddate=&pageitems=2000000000&season1={season}&season={season}"
        # Navegar a la página
        driver.get(url)

        # Esperar a que el elemento esté presente
        WebDriverWait(driver, 10)

        # Obtener el HTML de la página
        page_content = driver.page_source

        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(page_content, "html.parser")

        # Buscar la tabla de bateo
        table_content = soup.find("div",{"class":"table-scroll"})
        table_content_tr = table_content.find_all("tr")
        
        l = []
        for tr in table_content_tr:
            td = tr.find_all('td')
            th = tr.find_all('th')
            row = [tr.text for tr in td]
            team  = [tr.text for tr in th]
            complete_row = team + row
            l.append(complete_row)
        l_df=pd.DataFrame(l[1:],columns=l[0])
        l_df["Year"]=season
        
        df_list.append(l_df)
    driver.quit()
    
    advanced_df = pd.concat(df_list, ignore_index=True)
    
    return advanced_df

In [13]:
df = Fangraphs_teams_season(2000,2024,1)

df

,#,Team,PAPA - Plate Appearances,BB%BB% - Walk Percentage (BB/PA),K%K% - Strikeout Percentage (SO/PA),BB/KBB/K - Walk to Strikeout Ratio (BB/SO),-- Line Break --,AVGAVG - Batting Average (H/AB),OBPOBP - On Base Percentage,SLGSLG - Slugging Percentage,...,"UBRUBR - Ultimate Base Running in runs above average, does not include SB or CS",wGDPwGDP - Ground into Double Play Runs above average,XBRXBR - Statcast baserunning above average,wSBwSB - Stolen Bases and Caught Stealing runs above average,-- Line Break --,wRCwRC - Runs Created based wOBA,wRAAwRAA - Runs Above Average based on wOBA,wOBAwOBA - Weighted On Base Average (Linear Weights),wRC+wRC+ - Runs per PA scaled where 100 is average; both league and park adjusted; based on wOBA,Year
0,1,SFG,6418,11.0%,16.1%,0.69,,.278,.362,.472,...,,,,-1.3,,942,100.0,.359,112,2000
1,2,CLE,6512,10.5%,16.2%,0.65,,.288,.367,.470,...,,,,7.8,,987,132.3,.364,109,2000
2,3,OAK,6432,11.7%,18.0%,0.65,,.270,.360,.458,...,,,,1.9,,931,87.0,.356,109,2000
3,4,SEA,6444,12.0%,16.7%,0.72,,.269,.361,.442,...,,,,-0.5,,907,61.1,.352,107,2000
4,5,ANA,6373,9.5%,16.1%,0.59,,.280,.352,.472,...,,,,-4.5,,920,83.7,.356,106,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,26,CIN,5970,8.3%,24.4%,0.34,,.231,.305,.388,...,-2.4,2.3,3.1,12.6,,666,-32.4,.303,87,2024
356,27,MIA,6048,6.7%,23.3%,0.29,,.244,.300,.378,...,-0.7,-4.0,-0.1,-3.3,,637,-70.9,.296,86,2024
357,28,PIT,6086,8.0%,24.7%,0.32,,.234,.301,.371,...,12.2,3.4,1.1,2.4,,637,-74.9,.295,86,2024
358,29,COL,6026,7.5%,26.8%,0.28,,.242,.304,.400,...,1.6,-11.2,-0.2,-6.0,,683,-22.2,.306,82,2024


In [14]:
df.rename(columns={"BB%BB% - Walk Percentage (BB/PA)":"W rate",
                  "K%K% - Strikeout Percentage (SO/PA)":"K rate",
                  "BABIPBABIP - Batting Average on Balls in Play":"BABIP",
                   "wOBAwOBA - Weighted On Base Average (Linear Weights)":"wOBA"}, inplace= True)

In [15]:
Advanced_batting_df = df[["Team","W rate", "K rate", "BABIP", "wOBA","Year"]]

Advanced_batting_df.head()

,Team,W rate,K rate,BABIP,wOBA,Year
0,SFG,11.0%,16.1%,.303,.359,2000
1,CLE,10.5%,16.2%,.318,.364,2000
2,OAK,11.7%,18.0%,.300,.356,2000
3,SEA,12.0%,16.7%,.299,.352,2000
4,ANA,9.5%,16.1%,.303,.356,2000


In [16]:
Advanced_batting_df["Team"].value_counts().index

Index(['SFG', 'TEX', 'MIL', 'COL', 'PHI', 'MIN', 'ARI', 'CHC', 'KCR', 'PIT',
       'SDP', 'CLE', 'CIN', 'ATL', 'BOS', 'BAL', 'LAD', 'DET', 'TOR', 'NYY',
       'STL', 'HOU', 'NYM', 'CHW', 'SEA', 'OAK', 'WSN', 'LAA', 'FLA', 'TBD',
       'TBR', 'MIA', 'ANA', 'MON'],
      dtype='object', name='Team')

In [17]:
# Converting columns to integer format
Advanced_batting_df["W rate"] = (Advanced_batting_df["W rate"].str.rstrip('%').astype(float)) / 100  # Convert percentage to integer
Advanced_batting_df["K rate"] = Advanced_batting_df["K rate"].str.rstrip('%').astype(float) / 100  # Convert percentage to integer
Advanced_batting_df["BABIP"] = Advanced_batting_df["BABIP"].astype(float)  # Convert decimal to integer
Advanced_batting_df["wOBA"] = Advanced_batting_df["wOBA"].astype(float)  # Convert decimal to integer

C:\Users\PC\AppData\Local\Temp\ipykernel_59412\236093343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Advanced_batting_df["W rate"] = (Advanced_batting_df["W rate"].str.rstrip('%').astype(float)) / 100  # Convert percentage to integer
C:\Users\PC\AppData\Local\Temp\ipykernel_59412\236093343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Advanced_batting_df["K rate"] = Advanced_batting_df["K rate"].str.rstrip('%').astype(float) / 100  # Convert percentage to integer
C:\Users\PC\AppData\Local\Temp

In [18]:
Advanced_batting_df.dtypes

Team       object
W rate    float64
K rate    float64
BABIP     float64
wOBA      float64
Year        int32
dtype: object

In [19]:
Advanced_batting_df.head()

,Team,W rate,K rate,BABIP,wOBA,Year
0,SFG,0.110,0.161,0.303,0.359,2000
1,CLE,0.105,0.162,0.318,0.364,2000
2,OAK,0.117,0.180,0.300,0.356,2000
3,SEA,0.120,0.167,0.299,0.352,2000
4,ANA,0.095,0.161,0.303,0.356,2000


In [20]:
Advanced_batting_df.to_csv("Advanced_batting.csv")